# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium

In [3]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [4]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os
import torch

In [5]:
import gymnasium as gym
from gym import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [6]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import SubprocVecEnv

In [7]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [8]:
import plotly.express as px
import plotly.graph_objects as go

In [9]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Используем GPU
    print("CUDA доступна. Работаем на GPU.")
else:
    device = torch.device("cpu")   # Используем CPU
    print("CUDA не доступна. Работаем на CPU.")


CUDA доступна. Работаем на GPU.


# Open and Preprocessing Data

## Description of changes

Отличия от Дефолта:
- Изменили формулу награды, теперь расчет выполняется исходя из будущих изменений, а не предыдущих
- Убрали штраф за пересечение границы в половину стартового баланса

### Обозначения:

- `balance` — текущий баланс агента (денежные средства).
- `tokens_held` — количество токенов (активов), которыми владеет агент.
- `current_step` — текущий временной шаг.
- `Close_orig` — исходная цена закрытия на текущем шаге.
- `future_price` — будущая цена закрытия, извлекаемая из текущей строки данных:
  $$
  \text{future\_price} = \text{df.loc[current\_step, 'Close\_orig']}
  $$
- `fut_net_worth` — будущая чистая стоимость агента.
- `net_worth` — текущая чистая стоимость агента.
- `initial_balance` — начальный баланс агента.
- `reward` — награда агента за текущий шаг.
- `action` — действие, предпринятое агентом.
- `go_beyond` — флаг, указывающий на выход за пределы допустимых границ.
- `terminated` — флаг, указывающий на завершение эпизода.

### 1. **Расчёт будущей чистой стоимости**

Будущая чистая стоимость агента вычисляется по формуле:

$$
\text{fut\_net\_worth} = \text{balance} + (\text{tokens\_held} \times \text{future\_price})
$$

Это сумма текущего баланса и стоимости всех токенов по будущей цене.

### 2. **Изменение чистой стоимости**

Изменение чистой стоимости относительно текущей рассчитывается следующим образом:

$$
\text{net\_worth\_change} = \left( \frac{\text{fut\_net\_worth} \times 100}{\text{net\_worth}} \right) - 100
$$

Это процентное изменение чистой стоимости агента относительно текущей чистой стоимости.

### 3. **Изменение цены актива**

Изменение цены актива относительно текущей цены:

$$
\text{price\_change} = \left( \frac{\text{future\_price} \times 100}{\text{current\_price}} \right) - 100
$$

Это процентное изменение цены актива.

### 4. **Изменение относительно начального баланса**

Отношение текущей чистой стоимости к начальному балансу:

$$
\text{initial\_change} = \frac{\text{net\_worth}}{\text{initial\_balance}}
$$

Это показывает, насколько текущая чистая стоимость отличается от начального баланса.

### 5. **Корректировка награды в зависимости от изменения цены**

- **Если цена увеличилась** ($\text{price\_change} > 0$):

  $$ 
  \text{reward} += \text{net\_worth\_change} - \frac{\left| \left| \text{net\_worth\_change} \right| - \left| \text{price\_change} \right| \right|}{2}
  $$

- **Если цена уменьшилась** ($\text{price\_change} \leq 0$):

  $$
  \text{reward} += \text{net\_worth\_change} + \frac{\left| \left| \text{net\_worth\_change} \right| - \left| \text{price\_change} \right| \right|}{2}
  $$

Здесь учитывается разница между изменением чистой стоимости агента и изменением цены актива.

### 6. **Дополнительная корректировка награды в зависимости от начального баланса**

- **Если текущая чистая стоимость больше начального баланса** ($\text{net\_worth} > \text{initial\_balance}$):

  $$
  \text{reward} += \text{initial\_change}
  $$

- **Если текущая чистая стоимость меньше начального баланса** ($\text{net\_worth} \leq \text{initial\_balance}$):

  $$
  \text{reward} += - (1 - \text{initial\_change})
  $$

Это стимулирует агента увеличивать свою чистую стоимость относительно начального баланса.

### 7. **Штрафы**

- **За бездействие**: если агент не предпринимает действий ($\text{action} = 0$), то накладывается штраф:

  $$
  \text{reward} -= 1
  $$

- **За выход за границы**: если флаг `go_beyond` установлен ($\text{go\_beyond} \neq 0$), то накладывается дополнительный штраф:

  $$
  \text{reward} -= 5
  $$

### 8. **Завершение эпизода при значительном снижении чистой стоимости**

Если текущая чистая стоимость агента падает ниже 50% от начального баланса:

$$
\text{Если\ } \text{net\_worth} < 0.5 \times \text{initial\_balance}, \text{\ то\ } \text{terminated} = \text{True}
$$

Это означает, что агент достиг критического уровня убытков, и эпизод завершается (применяется только в режиме обучения, когда `mode == "train"`).

## Settings

In [17]:
window_length = 48
nb_steps = 80000

model_num = 2
data_num = 1

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed= seed)

## Open Data

In [18]:
# Загрузка данных
df = pd.read_csv(Path["dataset"](model_num, data_num))
df.shape

(103954, 26)

In [19]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [20]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25], dtype=int64)

In [21]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
1     4388
23    4388
18    4388
0     4387
5     4387
16    4384
15    4384
6     4379
25    4372
19    4371
4     4367
14    4342
13    4334
20    4314
3     4313
8     4309
22    4219
9     4207
21    4093
24    3602
11    3589
10    3582
17    3120
7     2957
2     2931
12    1847
Name: count, dtype: int64

In [22]:
def split_by_asset_ids(df: pd.DataFrame, test_asset_ids: list):
    test_df = df[df['Asset_ID_encoded'].isin(test_asset_ids)]
    train_df = df[~df['Asset_ID_encoded'].isin(test_asset_ids)]
    return train_df, test_df

In [23]:
test_asset_ids = [21, 22, 23, 24, 25]
train_df, test_df = split_by_asset_ids(df = df, test_asset_ids = test_asset_ids)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (83280, 26)
Testing data shape: (20674, 26)


# Game Rule

In [24]:
class TradingEnv(gym.Env):
    """
    Environment for training an agent to trade on the exchange using a continuous action space.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self, df, mode):
        super(TradingEnv, self).__init__()

        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length
        self.mode = mode # test or train

        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        num_features = len(self.df.columns) - 1 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0
        self.go_beyond = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'] - 1)
        print(self.now_token)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'tokens_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
            "cnt_go_beyond":[],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {asset_ids[0]:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = seed, options=None, reset_hist=False):
        super().reset(seed= seed)
        self.logger.info("Environment reset")

        # Existing reset logic
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        # Move to the next asset's starting index
        self.now_token += 1
        if self.now_token not in self.asset_start_indices:
            if self.mode == "train":
                self.now_token = list(self.asset_start_indices.keys())[0]  # Loop back to the first asset
            else:
                return None, None

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        # Reset hist only if reset_hist is True
        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'tokens_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
                "cnt_go_beyond":[],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]
        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        terminated = False
        truncated = False
        self.prev_token = self.now_token

        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.current_step += 1  # Move to the next time step
        
        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                self.now_token -=1
                terminated = True

        if not terminated:
            self.future_price = self.df.loc[self.current_step, 'Close_orig'] 
            self.fut_net_worth = self.balance + self.tokens_held * self.future_price

            if self.fut_net_worth != 0:
                net_worth_change = (self.fut_net_worth * 100 / self.net_worth) - 100
                price_change = (self.future_price * 100 / self.current_price) - 100
                initial_change = self.net_worth / self.initial_balance
                
                if price_change > 0:
                    self.reward += net_worth_change - (abs(abs(net_worth_change) - abs(price_change)) / 2)
                else:
                    self.reward += net_worth_change + (abs(abs(net_worth_change) - abs(price_change)) / 2)

                if self.net_worth > self.initial_balance:
                    self.reward += initial_change
                else:
                    self.reward += - (1 - initial_change)
                
                if action == 0:
                    self.reward -= 1
                
                if self.go_beyond != 0:
                    self.reward -= 5
                
                self.logger.info(f"step: {self.current_price - 1}, net_worth_change: {net_worth_change}, price_change: {price_change}, initial_change: {initial_change}, reward: {self.reward}")

                if self.net_worth < self.initial_balance * 0.5: # Только во время тренировки штрафуем за проеб половины баланса
                    if self.mode == "train":
                        self.logger.info("Net worth dropped below 50% of initial balance.")
                        terminated = True
            else:
                self.logger.info("fut_net_worth == 0")

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step - 1)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["tokens_held"].append(self.tokens_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)
        self.hist["cnt_go_beyond"].append(self.go_beyond)

        return obs, self.reward, terminated, truncated, info


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.tokens_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)

        self.net_worth = self.balance + self.tokens_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.tokens_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.go_beyond = 1
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.tokens_held:
            shares_to_sell = self.tokens_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.tokens_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.go_beyond = -1
            self.logger.info("No shares to sell.")


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.tokens_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [25]:
train_env = TradingEnv(train_df, mode = "train")

{0: 0, 1: 4387, 2: 8775, 3: 11706, 4: 16019, 5: 20386, 6: 24773, 7: 29152, 8: 32109, 9: 36418, 10: 40625, 11: 44207, 12: 47796, 13: 49643, 14: 53977, 15: 58319, 16: 62703, 17: 67087, 18: 70207, 19: 74595, 20: 78966}
-1


In [26]:
test_env = TradingEnv(test_df, mode = "test")

{21: 0, 22: 4093, 23: 8312, 24: 12700, 25: 16302}
20


In [27]:
# check_env(train_env)

# Train Model

## Settings

In [ ]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=512,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/",  # Directory for TensorBoard logs
    seed= seed,
)

In [22]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model"](model_num, data_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [ ]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        
    # callback=eval_callback   
    progress_bar= True,
)

In [24]:
model.save(Path["model"](model_num, data_num))

In [ ]:
train_hist = train_env.hist
train_hist_df = pd.DataFrame(train_hist)
print(len(train_hist["action"]))

In [ ]:
unique_values = train_hist_df['token'].unique()
unique_values

# Visualisaton

## Functions

In [48]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()

In [49]:
def plot_reward_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['reward'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['reward'], mode='lines', name='Reward'))

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Reward = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Reward and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Reward')

    # Показать график
    fig.show()

In [50]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()

In [51]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()
    fig.write_html("plot_1.html")

In [52]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()

In [53]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()

## Plots

In [ ]:
token = 20
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [ ]:
train_hist_df

In [ ]:
train_hist_df.describe()

# Test Model

## Test

In [34]:
model = SAC.load(Path["model"](model_num, data_num))

In [35]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist
        if info is None:
            break

In [36]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

20433


In [37]:
test_hist_df

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value,cnt_go_beyond
0,48,14.580618,999.015565,43165612,21,0.000023,-2.479920,0.985419,0,0.000000,0
1,49,0.129718,974.051405,43815084,21,0.000022,-0.007957,0.991104,0,0.000000,0
2,50,0.000732,974.226536,43820880,21,0.000022,-0.700476,0.994489,0,0.000000,0
3,51,0.000025,967.653404,43820912,21,0.000022,0.665054,0.993835,0,0.000000,0
4,52,0.000003,974.401824,43820913,21,0.000022,-0.884566,0.989902,0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...
20428,20668,321.911878,990.971386,1,25,669.059508,-4.813336,-0.807426,21,6286.389665,-1
20429,20669,321.911878,993.524952,1,25,671.613074,-5.230556,-0.628353,21,6286.389665,-1
20430,20670,321.911878,990.596934,1,25,668.685056,-5.223502,-0.493866,21,6286.389665,-1
20431,20671,321.911878,987.803736,1,25,665.891859,-4.972123,-0.266151,21,6286.389665,-1


In [38]:
unique_values = test_hist_df['token'].unique()
unique_values

array([21, 22, 23, 24, 25], dtype=int64)

## Plots

In [54]:
token = 25
plot_token_data(df = test_hist_df, token = token)
plot_reward_data(df = test_hist_df, token = token)
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [40]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - hist_df: DataFrame containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall averages for each column
    averages = report_df.mean(numeric_only=True)
    averages['Asset_ID'] = 'Average'  # Mark row as average

    # Append averages row to the DataFrame using pd.concat
    report_df = pd.concat([report_df, pd.DataFrame([averages])], ignore_index=True)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

- Asset_ID: Уникальный идентификатор актива (из столбца token), для которого рассчитываются метрики.

- Total Profit: Общий финансовый результат (прибыль или убыток) по данному активу. Рассчитывается как разница между конечной чистой стоимостью (net_worth) и начальным балансом (initial_balance).

- ROI (%): Доходность инвестиций (Return on Investment) в процентах. Показывает процентный прирост (или убыток) от начальной суммы баланса.

- Sharpe Ratio: Коэффициент Шарпа. Оценивает отношение доходности к риску (волатильности). Чем выше коэффициент Шарпа, тем лучше риск-корректированная доходность стратегии.

- Max Drawdown (%): Максимальная просадка в процентах. Это максимальное снижение стоимости актива от его исторического максимума. Отражает риски стратегии, связанные с падением стоимости.

- Win Rate (%): Процент прибыльных сделок. Это отношение количества прибыльных сделок к общему количеству сделок по активу, умноженное на 100.

- Buy-and-Hold Profit: Прибыль при стратегии "купить и держать". Показывает, сколько можно было бы заработать, если просто купить актив в начале и держать его до конца периода тестирования.

- Buy-and-Hold ROI (%): Доходность при стратегии "купить и держать". Процентный прирост от начальной цены актива, если его просто держать до конца периода.

- Ideal Profit: Идеальная прибыль. Это гипотетическая максимальная прибыль, которую можно было бы получить, если бы купили актив по минимальной цене и продали по максимальной цене за период.

- Ideal ROI (%): Идеальная доходность. Процентный прирост при идеальной стратегии, где покупка происходит по минимальной цене, а продажа — по максимальной.

- Asset Price Change (%): Изменение цены актива в процентах за период. Это процентное изменение цены от начальной до конечной за период тестирования.

In [41]:
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_13516\3864715686.py:85: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [42]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,21,-540.175580,-54.017558,-0.227290,-63.034744,47.218789,-279.352882,-27.935288,1596.459700,159.645970,-27.935288
1,22,74345.405975,7434.540598,0.771092,-73.251357,49.604411,253802.131121,25380.213112,293446.958188,29344.695819,25380.213112
2,23,-580.685035,-58.068503,0.072603,-85.656990,49.423963,-152.651940,-15.265194,9635.856272,963.585627,-15.265194
3,24,-338.776640,-33.877664,0.083608,-61.527881,49.634215,594.470161,59.447016,2103.489180,210.348918,59.447016
4,25,-11.674239,-1.167424,0.072537,-54.012511,49.525792,233.832793,23.383279,2790.670479,279.067048,23.383279
5,Average,14574.818896,1457.481890,0.154510,-67.496697,49.081434,50839.685851,5083.968585,61914.686764,6191.468676,5083.968585


In [43]:
test_hist_df.describe()

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value,cnt_go_beyond
count,20433.000000,2.043300e+04,20433.000000,2.043300e+04,20433.000000,20433.000000,20433.000000,20433.000000,2.043300e+04,20433.000000,20433.000000
mean,10359.856702,3.362978e+02,1824.245678,5.839982e+06,22.997015,78.111095,-3.096301,0.549345,1.145008e+08,13185.207296,0.615083
std,5965.411689,2.746553e+03,6132.455513,1.338258e+07,1.419961,154.761422,6.995115,0.600513,2.921292e+08,18471.204070,0.788326
min,48.000000,1.492709e-07,245.674375,0.000000e+00,21.000000,0.000011,-21.641873,-0.999994,0.000000e+00,0.000000,-1.000000
25%,5204.000000,1.003810e-02,551.095268,2.000000e+00,22.000000,0.008798,-5.596092,0.252757,5.020000e+02,3735.250191,1.000000
50%,10360.000000,3.254145e+00,733.554022,1.251000e+03,23.000000,0.272110,-4.751787,0.885553,2.997900e+04,9208.765532,1.000000
75%,15516.000000,2.608574e+02,989.800070,1.977310e+05,24.000000,8.408479,-1.851503,0.982144,3.612496e+06,18719.790832,1.000000
max,20672.000000,7.534465e+04,85411.704623,4.382091e+07,25.000000,674.820251,81.933701,0.999968,1.316063e+09,331170.235594,1.000000


In [44]:
test_hist_df["cnt_go_beyond"].value_counts()

cnt_go_beyond
 1    16498
-1     3930
 0        5
Name: count, dtype: int64

In [45]:
report_df.to_csv(Path["reports"](model_num, data_num), index= False)

In [46]:
overall_metrics

{'Total Profit': 87448.91337713393,
 'ROI (%)': 1748.9782675426784,
 'Sharpe Ratio': 0.15450991729185026,
 'Win Rate (%)': 49.0814340833281,
 'Buy-and-Hold Profit': 305038.1151049347,
 'Buy-and-Hold ROI (%)': 6100.762302098695}